##  Project: Statistical Analysis of Telecom Plans (Megaline)

---

### **Context and Primary Objective**

You are an analyst for the telecommunications company **Megaline**, which offers two prepaid plans: **Surf** and **Ultimate**. The commercial department needs to identify which plan generates **more revenue** in order to **adjust the advertising budget**.

Your task is to perform a **preliminary analysis** of customer behavior using a sample of **500 Megaline clients**. The focus is to analyze **usage behavior** (minutes, messages, and data) and determine, through **statistical tests**, which prepaid plan generates **more revenue, on average**.

---

### **Plan Details and Billing Rules**

**Note:** Megaline rounds seconds to minutes and megabytes to gigabytes. For calls, each individual call is **rounded up** (even if a call lasted only one second, it is counted as one full minute). For web traffic, the **monthly total is rounded up** (e.g., 1,025 MB is billed as 2 GB).

#### **Surf Plan**

* **Monthly Fee:** **$20**
* **Included Package:** 500 minutes, 50 text messages, and 15 GB of data.
* **Overage Charges:**
    * 1 minute: **$0.03**
    * 1 text message: **$0.03**
    * 1 GB of data: **$10**

#### **Ultimate Plan**

* **Monthly Fee:** **$70**
* **Included Package:** 3,000 minutes, 1,000 text messages, and 30 GB of data.
* **Overage Charges:**
    * 1 minute: **$0.01**
    * 1 text message: **$0.01**
    * 1 GB of data: **$7**
```

# 1. Which Plan Performs Better?



## Introduction

In [115]:
# Libraries for Data Manipulation
import pandas as pd
import numpy as np

# Libraries for Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for Statistical Testing
from scipy import stats
from scipy.stats import ttest_ind

pd.set_option('display.max_columns',None)

# Load the Data

In [116]:
# Load Data Files into Separate DataFrames
calls = pd.read_csv('datasets/megaline_calls.csv')
internet = pd.read_csv('datasets/megaline_internet.csv')
messages = pd.read_csv('datasets/megaline_messages.csv')
plans = pd.read_csv('datasets/megaline_plans.csv')
users = pd.read_csv('datasets/megaline_users.csv')


 ## 1.3 Prepare the Data

The data in this project is divided into several tables. Each table was explored to gain an initial understanding of the dataset. Necessary corrections were applied individually to ensure accuracy and consistency before proceeding with furth

# 1.4. Plans

In [117]:
#Print General Information / Summary of the Plans DataFrame
print(plans.info())
print("\nResumo estatístico:")
print(plans.describe())

print("\nVisualização das primeiras linhas:")
print(plans.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 260.0+ bytes
None

Resumo estatístico:
       messages_included  mb_per_month_included  minutes_included  \
count           2.000000               2.000000          2.000000   
mean          525.000000           23040.000000       1750.000000   
std           671.751442           10861.160159       1767.766953   
min        

In [118]:
#Print a Sample of the Plans Data
plans.sample(frac=1)
print(plans)

   messages_included  mb_per_month_included  minutes_included  \
0                 50                  15360               500   
1               1000                  30720              3000   

   usd_monthly_pay  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  
1               70           7             0.01            0.01  ultimate  


Data Quality Check – Plans Table

There are no missing values in the dataset, and the column plan_name contains the correct values “surf” and “ultimate,” which match the project description. The numeric columns are also consistent: usd_monthly_pay equals $20 and $70, while minutes_included, messages_included, and mb_per_month_included align with the described package limits. The additional pricing values are correct as well.

Some improvements could be made for clarity and consistency. The column usd_monthly_pay should be renamed to usd_monthly_fee, the values in plan_name should be standardized to lowercase, and the units in mb_per_month_included should be converted from megabytes to gigabytes.

##  Correct the Data

In [119]:
#Renaming the column to match the data dictionary
plans.rename(columns={'usd_monthly_pay': 'usd_monthly_fee'}, inplace=True)

#Converting values from megabytes to gigabytes (1 GB = 1024 MB)
plans['mb_per_month_included'] = plans['mb_per_month_included'] / 1024

print(plans)

   messages_included  mb_per_month_included  minutes_included  \
0                 50                   15.0               500   
1               1000                   30.0              3000   

   usd_monthly_fee  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  
1               70           7             0.01            0.01  ultimate  


## Enrich the Data

In [120]:
plans['plan_name'] = plans['plan_name'].str.lower()
print(plans)

   messages_included  mb_per_month_included  minutes_included  \
0                 50                   15.0               500   
1               1000                   30.0              3000   

   usd_monthly_fee  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  
1               70           7             0.01            0.01  ultimate  


## Users

In [121]:
#General Information about the 'users' DataFrame
print(users.info())

# Displaying the first rows to understand the structure of the data
print("\nAmostra de dados:")
print(users.head())

#Checking unique values for plans and cities
print("\nPlanos disponíveis:")
print(users['plan'].unique())

print("\nCidades representadas:")
print(users['city'].unique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB
None

Amostra de dados:
   user_id first_name  last_name  age                                   city  \
0     1000   Anamaria      Bauer   45  Atlanta-Sandy Springs-Roswell, GA MSA   
1     1001     Mickey  Wilkerson   28        Seattle-Tacoma-Bellevue, WA MSA   
2     1002     Carlee    Hoffman   36   Las Vegas-Henderson-Paradise, NV MSA   
3     1003   Reynaldo    Jenkins   52                          Tulsa, OK MSA   
4     1004    Leonila

In [122]:
# Print a sample of the users data
users.sample(5)
print(users)


     user_id first_name   last_name  age  \
0       1000   Anamaria       Bauer   45   
1       1001     Mickey   Wilkerson   28   
2       1002     Carlee     Hoffman   36   
3       1003   Reynaldo     Jenkins   52   
4       1004    Leonila    Thompson   40   
..       ...        ...         ...  ...   
495     1495      Fidel      Sharpe   67   
496     1496      Ariel    Shepherd   49   
497     1497      Donte     Barrera   49   
498     1498       Scot  Williamson   51   
499     1499      Shena     Dickson   37   

                                          city    reg_date      plan  \
0        Atlanta-Sandy Springs-Roswell, GA MSA  2018-12-24  ultimate   
1              Seattle-Tacoma-Bellevue, WA MSA  2018-08-13      surf   
2         Las Vegas-Henderson-Paradise, NV MSA  2018-10-21      surf   
3                                Tulsa, OK MSA  2018-01-28      surf   
4              Seattle-Tacoma-Bellevue, WA MSA  2018-05-23      surf   
..                                     

The columns reg_date and churn_date are stored as objects, but since they represent dates, they should be converted to the datetime format. The column churn_date contains many missing values (NaN), which indicates that most users are still active. This is consistent with the data dictionary, so no correction is required.

### Correct the Data

## Calls

In [123]:
#Print General Information / Summary of the Calls DataFrame
print(calls.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB
None


In [124]:
#Print a sample of the calls data
print("\nAmostra de dados:")
print(calls.head())


Amostra de dados:
         id  user_id   call_date  duration
0   1000_93     1000  2018-12-27      8.52
1  1000_145     1000  2018-12-27     13.66
2  1000_247     1000  2018-12-27     14.48
3  1000_309     1000  2018-12-28      5.76
4  1000_380     1000  2018-12-30      4.22


### Correct the Data

In [125]:
calls['call_date'] = pd.to_datetime(calls['call_date'])
print(calls)

              id  user_id  call_date  duration
0        1000_93     1000 2018-12-27      8.52
1       1000_145     1000 2018-12-27     13.66
2       1000_247     1000 2018-12-27     14.48
3       1000_309     1000 2018-12-28      5.76
4       1000_380     1000 2018-12-30      4.22
...          ...      ...        ...       ...
137730  1499_199     1499 2018-11-21      8.72
137731  1499_200     1499 2018-10-20     10.89
137732  1499_201     1499 2018-09-21      8.12
137733  1499_202     1499 2018-10-10      0.37
137734  1499_203     1499 2018-12-29     13.86

[137735 rows x 4 columns]


In [126]:
# Contagem de users por plano
plan_counts = users['plan'].value_counts().reset_index()
plan_counts.columns = ['Plano', 'Quantidade de Usuários']

print(plan_counts)

      Plano  Quantidade de Usuários
0      surf                     339
1  ultimate                     161


## Messages

In [127]:
# Print General Information / Summary of the Messages DataFrame
print(messages.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None


In [128]:
# Print a sample of the messages data
print(messages.head()) 

         id  user_id message_date
0  1000_125     1000   2018-12-27
1  1000_160     1000   2018-12-31
2  1000_223     1000   2018-12-31
3  1000_251     1000   2018-12-27
4  1000_255     1000   2018-12-26


In [129]:
messages['message_date'] = pd.to_datetime(messages['message_date'])
print(messages)

             id  user_id message_date
0      1000_125     1000   2018-12-27
1      1000_160     1000   2018-12-31
2      1000_223     1000   2018-12-31
3      1000_251     1000   2018-12-27
4      1000_255     1000   2018-12-26
...         ...      ...          ...
76046  1497_526     1497   2018-12-24
76047  1497_536     1497   2018-12-24
76048  1497_547     1497   2018-12-31
76049  1497_558     1497   2018-12-24
76050  1497_613     1497   2018-12-23

[76051 rows x 3 columns]


In [130]:
messages['message_count'] = messages.groupby('user_id')['id'].transform('count')
print(messages)

             id  user_id message_date  message_count
0      1000_125     1000   2018-12-27             11
1      1000_160     1000   2018-12-31             11
2      1000_223     1000   2018-12-31             11
3      1000_251     1000   2018-12-27             11
4      1000_255     1000   2018-12-26             11
...         ...      ...          ...            ...
76046  1497_526     1497   2018-12-24             50
76047  1497_536     1497   2018-12-24             50
76048  1497_547     1497   2018-12-31             50
76049  1497_558     1497   2018-12-24             50
76050  1497_613     1497   2018-12-23             50

[76051 rows x 4 columns]


## internet

In [131]:
# Print General Information / Summary of the internet DataFrame
print(internet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB
None


In [132]:
#Print a sample of the internet traffic data
print(internet.head())

         id  user_id session_date  mb_used
0   1000_13     1000   2018-12-29    89.86
1  1000_204     1000   2018-12-31     0.00
2  1000_379     1000   2018-12-28   660.40
3  1000_413     1000   2018-12-26   270.99
4  1000_442     1000   2018-12-27   880.22


In [133]:
internet['session_date'] = pd.to_datetime(internet['session_date'])
internet['gb_used'] = internet['mb_used'] / 1024 
print(internet)

              id  user_id session_date  mb_used   gb_used
0        1000_13     1000   2018-12-29    89.86  0.087754
1       1000_204     1000   2018-12-31     0.00  0.000000
2       1000_379     1000   2018-12-28   660.40  0.644922
3       1000_413     1000   2018-12-26   270.99  0.264639
4       1000_442     1000   2018-12-27   880.22  0.859590
...          ...      ...          ...      ...       ...
104820  1499_215     1499   2018-10-20   218.06  0.212949
104821  1499_216     1499   2018-12-30   304.72  0.297578
104822  1499_217     1499   2018-09-22   292.75  0.285889
104823  1499_218     1499   2018-12-07     0.00  0.000000
104824  1499_219     1499   2018-12-24   758.31  0.740537

[104825 rows x 5 columns]


In [134]:
internet['day_of_week'] = internet['session_date'].dt.dayofweek
print(internet)

              id  user_id session_date  mb_used   gb_used  day_of_week
0        1000_13     1000   2018-12-29    89.86  0.087754            5
1       1000_204     1000   2018-12-31     0.00  0.000000            0
2       1000_379     1000   2018-12-28   660.40  0.644922            4
3       1000_413     1000   2018-12-26   270.99  0.264639            2
4       1000_442     1000   2018-12-27   880.22  0.859590            3
...          ...      ...          ...      ...       ...          ...
104820  1499_215     1499   2018-10-20   218.06  0.212949            5
104821  1499_216     1499   2018-12-30   304.72  0.297578            6
104822  1499_217     1499   2018-09-22   292.75  0.285889            5
104823  1499_218     1499   2018-12-07     0.00  0.000000            4
104824  1499_219     1499   2018-12-24   758.31  0.740537            0

[104825 rows x 6 columns]


## Study the plan conditions

In [135]:
#Print the plan conditions and make sure they are consistent
print(plans)

   messages_included  mb_per_month_included  minutes_included  \
0                 50                   15.0               500   
1               1000                   30.0              3000   

   usd_monthly_fee  usd_per_gb  usd_per_message  usd_per_minute plan_name  
0               20          10             0.03            0.03      surf  
1               70           7             0.01            0.01  ultimate  


## Aggregate the data by user



In [ ]:
# Calculate the number of calls made by each user per month and save the result

# 1. Make sure the 'call_date' column is in datetime format
calls['call_date'] = pd.to_datetime(calls['call_date'])

# 2. Create a 'month' column to extract the month and year from 'call_date'
calls['month'] = calls['call_date'].dt.to_period('M')

# 3. Aggregate the data to count the number of calls per user per month
calls_per_user_month = calls.groupby(['user_id', 'month']).size().reset_index(name='num_calls')

# 4. Save the result to a CSV file
calls_per_user_month.to_csv('calls_per_user_month.csv', index=False)

# 5. Print the result
print(calls_per_user_month.head())


In [ ]:
#Calculate the number of minutes spent by each user per month and save the result

# Convert the date to datetime
calls['call_date'] = pd.to_datetime(calls['call_date'])

# Create a month column
calls['call_month'] = calls['call_date'].dt.to_period('M').astype(str)

# Group by user and month, summing the minutes used
minutes_per_user_month = calls.groupby(['user_id', 'call_month'])['duration'].sum().reset_index()

#Rename the month column to maintain consistency
minutes_per_user_month.rename(columns={'call_month': 'month'}, inplace=True)

# Print the result
print(minutes_per_user_month.head())


In [ ]:
# Calculate the number of messages sent by each user per month and save the result
messages['message_date'] = pd.to_datetime(messages['message_date'])

# Create a Month Column
messages['message_month'] = messages['message_date'].dt.to_period('M').astype(str)

# Group por usuário e month, contando a quantidade de messages
messages_per_user_month = messages.groupby(['user_id', 'message_month'])['id'].count().reset_index()

# Display the Result
print(messages_per_user_month.head())

In [ ]:
# Calculate o volume de traffic de internet usado por cada usuário por month. Salve o resultado.

# Convert the Date to Datetime
internet['session_date'] = pd.to_datetime(internet['session_date'])

# Create a Month Column
internet['session_month'] = internet['session_date'].dt.to_period('M').astype(str)

# Group por usuário e month, somando o uso de Data
internet_for_month_sum = internet.groupby(['user_id', 'session_month'])['gb_used'].sum().reset_index()

# Display the Result
print(internet_for_month_sum.head())

In [ ]:
# Junte os Data de calls, minutes, messages e internet com base em user_id e month

minutes_per_user_month = minutes_per_user_month.rename(columns={
    'call_month': 'month',
    'minutes': 'minutes_used'
})

messages_per_user_month = messages_per_user_month.rename(columns={
    'message_month': 'month',
    'id': 'messages_sent'
})

internet_for_month_sum = internet_for_month_sum.rename(columns={
    'session_month': 'month',
    
})


In [ ]:
dados_uso = minutes_per_user_month \
    .merge(messages_per_user_month, on=['user_id', 'month'], how='outer') \
    .merge(internet_for_month_sum, on=['user_id', 'month'], how='outer') \
    .merge(users[['user_id', 'plan']], on='user_id', how='left')

dados_uso = dados_uso.rename(columns={'plan':'plan_name'})
dados_uso.head()

In [ ]:
print(dados_uso.isna().sum())
dados_uso['duration'] = dados_uso['duration'].fillna(0)
dados_uso['messages_sent'] = dados_uso['messages_sent'].fillna(0)
dados_uso['gb_used'] = dados_uso['gb_used'].fillna(0)
print(dados_uso.isna().sum())

In [ ]:
dados_uso = dados_uso.merge(plans,on='plan_name',how='left')
dados_uso['gb_per_month_included'] = dados_uso['mb_per_month_included'] / 1024
dados_uso.head()

In [ ]:
dados_uso['messages_exceeded'] = np.maximum(0, (dados_uso['messages_sent'] - dados_uso['messages_included']))
dados_uso['messages_exceeded_price'] = dados_uso['messages_exceeded'] * dados_uso['usd_per_message']
dados_uso.head()

In [ ]:
dados_uso['minutes_exceeded'] = np.maximum(0, (dados_uso['duration'] - dados_uso['minutes_included']))
dados_uso['minutes_exceeded_price'] = dados_uso['minutes_exceeded'] * dados_uso['usd_per_minute']
dados_uso.head()

In [ ]:
dados_uso['internet_exceeded'] = np.maximum(0, (dados_uso['gb_used'] - dados_uso['gb_per_month_included']))
dados_uso['internet_exceeded_price'] = dados_uso['internet_exceeded'] * dados_uso['usd_per_gb']
dados_uso.head()

In [ ]:
dados_uso['receita'] = (
    dados_uso['messages_exceeded_price'] 
    + dados_uso['minutes_exceeded_price']
    + dados_uso['internet_exceeded_price'] 
    + dados_uso['usd_monthly_fee']
) 
dados_uso.head()

In [ ]:
dados_uso['receita'].isna().sum()

## Study user behavior

### Calls

In [ ]:
# Compare the number of minutes used per month by users of each plan. Build a histogram.

# Group by plan and month, and calculate the average minutes used (using 'duration')
media_chamadas = dados_uso.groupby(['plan_name', 'month'])['duration'].mean().reset_index()

# Plot the chart
plt.figure(figsize=(14, 6))
sns.barplot(data=media_chamadas, x='month', y='duration', hue='plan_name')

plt.title(' Average Call Duration by Plan and Month ')
plt.xlabel('Month')
plt.ylabel('Average Call Duration (minutes)')
plt.xticks(rotation=45)
plt.legend(title='Plan')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the mean and variance of monthly call duration

# Calculate a mean e a variance da duration monthly das calls por plano
estatisticas_duracao = dados_uso.groupby(['plan_name', 'month'])['duration'].agg(['mean', 'var']).reset_index()

# Display as primeiras linhas do resultado
print(estatisticas_duracao)

In [ ]:

# Create a Boxplot to Visualize the Distribution of Monthly Call Duration


# Create o boxplot para visualizar a distribuição da duration das calls por plano e month
plt.figure(figsize=(14, 6))
sns.boxplot(data=dados_uso, x='month', y='duration', hue='plan_name')

# Ajustar título e rótulos
plt.title('Distribution of Call Duration by Plan and Month')
plt.xlabel('Month')
plt.ylabel('Call Duration (minutes)')
plt.legend(title='Plano')

# Display o gráfico
plt.tight_layout()
plt.show()

Surf plan users typically consume more minutes than expected, often leading to additional costs. Ultimate plan users, however, show more stable usage within their larger allowance, resulting in lower risk of overage fees.

The usage variability observed among 'Surf' users is higher and less predictable, while 'Ultimate' users maintain a consistent usage pattern. This monthly fluctuation in minute consumption across the board likely reflects evolving customer demand or the impact of promotional activities.

In [ ]:
# Compare o número de messages que os users de cada plano costumam enviar a cada month
# Group por plano e month, e Calculate a mean de messages enviadas
media_mensagens = dados_uso.groupby(['plan_name', 'month'])['messages_sent'].mean().reset_index()

# Plot the Chart
plt.figure(figsize=(14, 6))
sns.barplot(data=media_mensagens, x='month', y='messages_sent', hue='plan_name')

plt.title('Average Number of Messages Sent by Plan and Month')
plt.xlabel('Month')
plt.ylabel('Average Number of Messages Sent')
plt.xticks(rotation=45)
plt.legend(title='Plano')
plt.tight_layout()
plt.show()


In [ ]:
# Compare a quantidade de traffic de internet consumido pelos users por plano

# Group por plano e Calculate a mean de Data usados (em MB)
media_dados = dados_uso.groupby('plan_name')['gb_used'].mean().reset_index()

# Plot the Chart
plt.figure(figsize=(10, 6))
sns.barplot(data=media_dados, x='plan_name', y='gb_used')

plt.title('Average Internet Data Consumed by Plan ')
plt.xlabel('Plan')
plt.ylabel('Internet Data Consumed (MB)')
plt.tight_layout()
plt.show()

Surf plan users demonstrate a restrained and careful approach to data consumption, contrasting sharply with Ultimate plan users, who exhibit a more liberal usage pattern, leveraging their greater data freedom.

### internet

In [ ]:
# Calculate a mean e a variance do traffic de Data
media_dados = dados_uso.groupby('plan_name')['gb_used'].mean().reset_index()
variancia_dados = dados_uso.groupby('plan_name')['gb_used'].var().reset_index()

# Display as médias e variâncias
print("Média de dados consumidos por plano:")
print(media_dados)
print("\nVariância de dados consumidos por plano:")
print(variancia_dados)

In [ ]:
# Comparar a distribuição de Data consumidos por plano usando boxplot
# Group por month e plano, tirando a mean de uso de Data
dados_mensais = dados_uso.groupby(['month', 'plan_name'])['gb_used'].mean().reset_index()

# Gráfico
plt.figure(figsize=(14, 6))
sns.lineplot(x='month', y='gb_used', hue='plan_name', data=dados_mensais, marker='o')
plt.title('Average Monthly Internet Consumption by Plan')
plt.xlabel('Month')
plt.ylabel('Average Monthly Internet Consumption (MB)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

Data consumption behavior is clearly segmented by plan structure. Surf plan users tend to be more cautious and restrained with their data usage due to the stricter limit, directly aiming to avoid costly overage fees. In contrast, Ultimate plan users enjoy greater freedom to consume data, which can result in usage being more volatile and, in some cases, significantly higher than the Surf plan.



## Revenue

In [ ]:
dados_uso['user_id'] = dados_uso['user_id'].astype(int)
users['user_id'] = users['user_id'].astype(int)

In [ ]:
# Check for Cross-Group User Contamination
print(dados_uso.columns)
print(users.columns)

# Merge entre dados_uso e users
dados_uso = dados_uso.merge(users[['user_id', 'city']], on='user_id', how='left')

In [ ]:
dados_uso.sort_values('gb_used',ascending=False)

In [ ]:
# Calculate a mean e a variance da receita por plano
media_receita = dados_uso.groupby('plan_name')['receita'].mean().reset_index()
variancia_receita = dados_uso.groupby('plan_name')['receita'].var().reset_index()

# Display as médias e variâncias
print("Média de receita por plano:")
print(media_receita)
print("\nVariância de receita por plano:")
print(variancia_receita)

In [ ]:
# Compare Monthly Revenue Distribution by Plan (Boxplot)
plt.figure(figsize=(14, 6))
sns.boxplot(x='plan_name', y='receita', data=dados_uso)
plt.title('Monthly Revenue Distribution by Plan')
plt.xlabel('Plan')
plt.ylabel('Monthly Revenue')
plt.tight_layout()
plt.show()

The Ultimate plan offers superior financial predictability, delivering a higher mean revenue with significantly less monthly volatility. This stability suggests its customer base adheres to a consistent, premium usage pattern.

The Surf plan, while having a smaller revenue average, is characterized by high variance. Its revenue is dependent on the unpredictable nature of overage fees, reflecting a less uniform and more diversified user base.

## Statistical Hypothesis Testing

In [ ]:
# Segment Data by Plan Type
surf = dados_uso[dados_uso['plan_name'] == 'surf']['receita']
ultimate = dados_uso[dados_uso['plan_name'] == 'ultimate']['receita']

# Perform Independent Samples T-Test 
t_stat, p_value = stats.ttest_ind(surf, ultimate, equal_var=False)

print(f'Estatística t: {t_stat:.4f}')
print(f'Valor-p: {p_value:.4f}')

# Check the Hypothesis (Determine Significance)
alpha = 0.05
if p_value < alpha:
    print("❌ Rejeitamos a hipótese nula: as receitas médias são significativamente diferentes.")
else:
    print("✅ Não rejeitamos a hipótese nula: não há diferença estatisticamente significativa.")


In [ ]:
dados_uso.describe()

In [ ]:
cidades_ny_nj = ['New York-Newark-Jersey City, NY-NJ-PA MSA']

ny_nj = dados_uso[dados_uso['city'].isin(cidades_ny_nj)]['receita']
outras_cidades = dados_uso[~dados_uso['city'].isin(cidades_ny_nj)]['receita']

# Check if Groups Have Data
print(f'NY-NJ: {len(ny_nj)} usuários')
print(f'Outras cidades: {len(outras_cidades)} usuários')

In [ ]:
ny_nj

In [ ]:
outras_cidades

In [ ]:
cidades_ny_nj = ['New York-Newark-Jersey City, NY-NJ-PA MSA']

#Refine Data Subset
ny_nj = dados_uso[dados_uso['city'].isin(cidades_ny_nj)]['receita']
outras_cidades = dados_uso[~dados_uso['city'].isin(cidades_ny_nj)]['receita']

# Check if Groups Have Data
print(f'NY-NJ: {len(ny_nj)} usuários')
print(f'Outras cidades: {len(outras_cidades)} usuários')

In [ ]:
# Remover values ausentes (por segurança)
ny_nj = ny_nj.dropna()
outras_cidades = outras_cidades.dropna()

# Execute Welch's T-Test (Assuming Unequal Variances) 
t_stat, p_value = stats.ttest_ind(ny_nj, outras_cidades, equal_var=False)

print(f'Estatística t: {t_stat:.2f}')
print(f'Valor p: {p_value:.4f}')


if p_value < 0.05:
    print("✅ Há diferença significativa na receita média entre usuários de NY-NJ e os de outras regiões.")
else:
    print("❌ Não há diferença significativa na receita média entre usuários de NY-NJ e os de outras regiões.")

[Formule as hipóteses nula e alternativa, escolha o teste estatístico, escolha o valor alfa.]

In [ ]:
# Filter Data by Exact City Name Match
ny_nj = dados_uso[dados_uso['city'] == 'New York-Newark-Jersey City, NY-NJ-PA MSA']['receita'].dropna()
outras = dados_uso[dados_uso['city'] != 'New York-Newark-Jersey City, NY-NJ-PA MSA']['receita'].dropna()

# Check for Sufficient Sample Size
print(f"Receitas NY-NJ: {ny_nj.shape}")
print(f"Receitas outras regiões: {outras.shape}")

# Execute Welch's T-Test (for Samples with Unequal Variances)
if ny_nj.shape[0] > 0 and outras.shape[0] > 0:  # Verificando se existem dados suficientes para realizar o teste
    t_stat, p_val = ttest_ind(ny_nj, outras, equal_var=False)
    print("Estatística t:", t_stat)
    print("Valor p:", p_val)

    alpha = 0.05

    if p_val < alpha:
        print("Rejeitamos a hipótese nula: A receita média das duas regiões é significativamente diferente.")
    else:
        print("Falhamos em rejeitar a hipótese nula: Não há diferença significativa entre as receitas médias.")
else:
    print("Não há dados suficientes para realizar o teste.")

# Project Conclusion

This comparative analysis of the **Surf** and **Ultimate** plans revealed statistically significant differences in customer behavior and revenue contribution, providing a solid foundation for the company's future pricing and retention strategies.

---

# Usage Behavior and Revenue Pattern

The consumption pattern is clearly linked to the pricing structure of each plan:

* **Surf Plan:** Users demonstrate a more **cautious and predictable** consumption (both minutes and data), which results in a **more constant** and stable revenue stream for the company, with lower overall volatility.
* **Ultimate Plan:** The higher limits and greater freedom lead to **higher data usage** and a more **volatile** overall customer behavior. This generates **revenue peaks** and a greater spread (variance) in the total revenue collected.

# Statistical Analysis and Financial Performance

1.  **Average Revenue:** Statistical tests confirmed that the **mean revenue** generated by the **Ultimate plan is significantly different (and superior)** to the Surf plan. The Ultimate plan is the higher-value revenue generator.
2.  **Volatility:** Despite having a higher mean, the Ultimate plan's consumption freedom results in **greater revenue variability**, while the Surf plan offers a more predictable revenue base.

# Regional Hypothesis Test

The test to verify the difference in average revenue between the **NY-NJ region** and other areas **did not yield conclusive results**. This failure was due to the **insufficient data** (lack of a statistically robust sample size) for that specific geographical segmentation.

# Methodological Decisions and Assumptions

The final conclusions were based on a rigorous analytical process:

* **Statistical Test:** The **Independent Samples T-Test** (or Welch's T-Test, used for unequal variances) was employed to compare mean revenues between the groups, addressing the non-normal nature and outliers often found in revenue data.
* **Significance Level ($\alpha$):** A significance level of **$\alpha = 0.05$** was used for all statistical tests, ensuring a $95\%$ confidence level in the results.
* **Data Segmentation:** Customer samples were **correctly segmented** by plan type (*Surf* and *Ultimate*) prior to testing, ensuring the independence of the samples.